In [46]:
sc

<SparkContext master=local[*] appName=PySparkShell>

In [47]:
from pyspark import SparkContext

In [48]:
# Reading file into RDD
data = sc.textFile("./PageRank/input.txt").map(lambda line: line.split(" ")) \
    .filter(lambda line: len(line)>=1).map(lambda line: (line[0],line[1]))

In [49]:
data.collect()

[('1', '1'), ('1', '2'), ('2', '1'), ('2', '3'), ('3', '2')]

In [50]:
links = data.distinct().groupByKey().map(lambda x: (x[0],list(x[1])))

In [51]:
links.collect()

[('1', ['1', '2']), ('2', ['3', '1']), ('3', ['2'])]

In [52]:
weight = links.map(lambda x :(x[0],1.0))

In [53]:
weight.collect()

[('1', 1.0), ('2', 1.0), ('3', 1.0)]

In [54]:
def weightage(links,weight):
    num_links = len(links)
    weight_new = []
    for link in links :
        weight_new.append((link,weight/num_links))
    return weight_new

In [55]:
for i in range(10):
    weight_final = links.join(weight).flatMap(lambda x: weightage(x[1][0],x[1][1]))
    weight_updated = weight_final.reduceByKey(lambda x,y : x+y)

In [56]:
weight_sorted = weight_updated.sortByKey()

In [57]:
final_weights = weight_sorted.map(lambda y: (str(y[0]),str(y[1]))).collect()

In [58]:
print("final weights are: ")
print(final_weights[0][0],'\t',final_weights[0][1])
print(final_weights[1][0],'\t',final_weights[1][1])
print(final_weights[2][0],'\t',final_weights[2][1])

final weights are: 
1 	 1.0
2 	 1.5
3 	 0.5
